In [1]:
from datasets import load_dataset

ds = load_dataset("surrey-nlp/PLOD-filtered")

c:\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd

# Assuming your DatasetDict is named `dataset_dict`

# Function to process each dataset and join tokens into sentences
def process_dataset(dataset):
    return pd.DataFrame({
        'id': dataset['id'],
        'sentence': [' '.join(tokens) for tokens in dataset['tokens']]
    })

# Process each split
train_df = process_dataset(ds['train'])
validation_df = process_dataset(ds['validation'])
test_df = process_dataset(ds['test'])

# Merge the DataFrames into a single DataFrame (optional)
merged_df = pd.concat([train_df, validation_df, test_df], ignore_index=True)

# Display the merged DataFrame
print(merged_df.head())


  id                                           sentence
0  0  Alternatively , fibroblasts were plated sparse...
1  1  Study - specific risk ratios ( RRs ) and mean ...
2  2  Three faba bean genotypes , Aguadulce ( AD ) ,...
3  3  Neurological complications of cancer include d...
4  4  Reference values ( RVs ) of each analyte were ...


In [ ]:
merged_df.head()

,id,sentence
0,0,"Alternatively , fibroblasts were plated sparse..."
1,1,Study - specific risk ratios ( RRs ) and mean ...
2,2,"Three faba bean genotypes , Aguadulce ( AD ) ,..."
3,3,Neurological complications of cancer include d...
4,4,Reference values ( RVs ) of each analyte were ...


In [1]:
import re

def extractAbbrev(element):
    try:
        # Regex pattern to find fully uppercase words in parentheses
        pattern = r'\(([A-Z]+)\)'

        # Find all matches with their start positions
        matches = [(match.group(1), match.start()) for match in re.finditer(pattern, element)]
        
        # Function to remove HTML tags from a string
        def remove_html_tags(text):
            return re.sub(r'<[^>]*>', '', text)

        # Extract words based on the new condition
        result = []
        for word, position in matches:
            
            # Find the preceding content up to the word's position
            preceding_text = element[:position].split()
            
            # Extract words starting with the same letter as the abbreviation
            extracted_words = []
            for w in reversed(preceding_text):
                extracted_words.insert(0,w)  # Insert at the beginning of the list
                # Stop extracting if the word doesn't contain the first letter
                if w[0].lower() not in word.lower():
                    break

            result.append((word, [remove_html_tags(item) for item in extracted_words]))  # Remove HTML tags

    except Exception as e:
        print(f"Error: {e}")
        return []
    
    return result
def preprocessData(resultsAnnotation):
    for index in range(len(resultsAnnotation)):
        try :
            word = resultsAnnotation[index][0]
            listOfWords = resultsAnnotation[index][1]
            firstLetter = word[0]
            # Ensure listOfWords has words starting with the same letter as the abbreviation
            while listOfWords and listOfWords[0][0].lower() != firstLetter.lower():
                resultsAnnotation[index][1].pop(0)
        except:
            continue
    return resultsAnnotation
import json

def string_to_object(string):
    try:
        return json.loads(string)
    except json.JSONDecodeError as e:
        raise ValueError(f"Invalid JSON string: {e}")
def get_surrounding_sentence(text, abbreviation):
    try:
        # Find the position of the abbreviation in the text
        match = re.search(re.escape(abbreviation), text)
        if not match:
            return None

        # Start and end positions of the abbreviation
        start, end = match.start(), match.end()

        # Find the closest sentence boundaries
        before = text[:start].rfind('. ')
        before = before if before != -1 else 0
        after = text[end:].find('. ')
        after = after if after != -1 else len(text)

        # Extract the sentence
        surrounding_sentence = text[before + 2 : end + after].strip()  # +2 to skip the period and space
        return surrounding_sentence
    except Exception as e:
        print(f"Error in extracting sentence: {e}")
        return None
def string_to_object(string):
    try:
        return json.loads(string)
    except json.JSONDecodeError as e:
        raise ValueError(f"Invalid JSON string: {e}")
def make_valid_json(s):
    # Step 1: Replace single quotes around keys with double quotes
    s = re.sub(r"(?<=\{|,)\s*'([^']+)'\s*:", r'"\1":', s)

    # Step 2: Replace single quotes around values with double quotes
    s = re.sub(r":\s*'([^']*)'\s*(,|\})", r':"\1"\2', s)

    # Step 3: Replace single quotes around list elements with double quotes
    s = re.sub(r"\[\s*'([^']*)'\s*(?:,|\])", lambda m: f'["{m.group(1)}"' if m.group(0)[-1] == ']' else f'["{m.group(1)}",', s)
    s = re.sub(r'",\s*\'([^\']*)\'\s*\]', r'", "\1"]', s)

    # Step 4: Remove stray single quotes that might exist
    s = s.replace("'", '"')

    return s
def row_to_object(row):
    # Split the expansion into a list of words
    expansion_words = row['expansion']
    # Create the object
    obj = {
        'abbreviation': row['abbreviation'],
        'expansion': expansion_words,
        'sentence': row['sentence']
    }
    # Convert to string
    return str(obj)
import json

def fix_json_format(string):
    # Step 1: Fix the quote marks for JSON compliance (ensure double quotes)
    string = string.replace("'", '"')  # Replaces single quotes with double quotes
    
    # Step 2: Check if the string ends correctly with a closing square bracket if needed
    if string.endswith("]]"):
        string = string[:-2] + "]"  # Fix the extra closing square brackets
    
    # Step 3: Ensure the sentence field ends correctly (if needed)
    if ',"sentence"' in string:
        string = string.replace(',"sentence"', '],"sentence"')  # Correct position of sentence
    
    try:
        return json.loads(string)  # Try to parse it into a Python object
    except json.JSONDecodeError as e:
        raise ValueError(f"Invalid JSON string: {e}")
import codecs

def remove_escape_characters(input_string):
    # Remove escape sequences and non-ASCII characters (or any specific unwanted characters)
    cleaned_string = re.sub(r'\\x[0-9A-Fa-f]{2}', '', input_string)  # Removing hex escape sequences like \\x82
    cleaned_string = re.sub(r'[^\x00-\x7F]+', '', cleaned_string)  # Removing non-ASCII characters (like Ã³)
    
    return cleaned_string

In [ ]:
import re

def get_surrounding_sentence(text, abbreviation):
    try:
        # Find the position of the abbreviation in the text
        match = re.search(re.escape(abbreviation), text)
        if not match:
            return None

        # Start and end positions of the abbreviation
        start, end = match.start(), match.end()

        # Find the closest sentence boundaries
        before = text[:start].rfind('. ')
        before = before if before != -1 else 0
        after = text[end:].find('. ')
        after = after if after != -1 else len(text)

        # Extract the sentence
        surrounding_sentence = text[before + 2 : end + after].strip()  # +2 to skip the period and space
        return surrounding_sentence
    except Exception as e:
        print(f"Error in extracting sentence: {e}")
        return None


results = []

for i in range(len(merged_df)):
    try:
        # Process the current sentence and extract abbreviations
        sentence = merged_df.iloc[i]["sentence"].replace('( ', "(").replace(' )', ")")
        extracted_abbreviations = preprocessData(extractAbbrev(sentence))
        
        for element in extracted_abbreviations:
            # Check that both abbreviation and expansion are valid
            if len(element[0]) > 1 and len(element[1]) > 1:
                # Retrieve the sentence where the abbreviation appears
                surrounding_sentence = get_surrounding_sentence(sentence, element[0])
                if surrounding_sentence:
                    print(f"Sentence: {surrounding_sentence}")
                    print(f"Abbreviation: {element[0]}, Expansion: {element[1]}")
                    
                    # Append the abbreviation, expansion, and extracted sentence to the results
                    results.append({
                        "abbreviation": element[0],
                        "expansion": element[1],
                        "sentence": surrounding_sentence
                    })
    except Exception as e:
        print(f"Error processing row {i}: {e}")
        continue


In [10]:
import pandas as pd 
results=pd.DataFrame(results)
results.to_json("../abbreviationExpansionDataset/abbreviationsDataset.json")

In [20]:
results.columns

Index(['abbreviation', 'expansion', 'sentence'], dtype='object')

In [21]:
data=pd.read_json("../abbreviationExpansionDataset/abbreviationsDataset.json")

In [ ]:
import pandas as pd



# Create DataFrame
df = pd.DataFrame(data)

# Function to convert row to desired string format
def row_to_object(row):
    # Split the expansion into a list of words
    expansion_words = (row['expansion'])
    # Create the object
    obj = {
        'abbreviation': row['abbreviation'],
        'expansion': expansion_words,
        'sentence': row['sentence']
    }
    # Convert to string
    return str(obj)

# Apply the function to each row
df['object_string'] = df.apply(row_to_object, axis=1)

# Display the DataFrame
print(df[['object_string']])


In [32]:
results=[]
for i in range(len(df)):
    results.append(df.iloc[i]["object_string"])


In [33]:
results=pd.DataFrame(results)


In [38]:
results.to_csv("./abbreviationsDataset.csv",index=False)

In [10]:
import pandas as pd 
dx=pd.read_csv("plos/valid.csv")

In [11]:
dx.columns

Index(['ABSTRACT_ID', 'TEXT', 'LOCATION', 'LABEL'], dtype='object')

In [2]:
def process_row(row):
    abbreviation_word = row["TEXT"].split(" ")[row["LOCATION"]]
    abbreviation = abbreviation_word.upper()
    
    # Generate the expansion with proper capitalization
    expansion = [
        word.capitalize() for word in row["LABEL"].split(" ")
    ]
    
    # Replace the abbreviation in the sentence with the expanded form
    expanded_form = f"{' '.join(expansion)} ({abbreviation})"
    sentence = row["TEXT"].replace(abbreviation_word, expanded_form)
    
    return {
        "abbreviation": abbreviation,
        "expansion": expansion,
        "sentence": sentence
    }


In [ ]:
results=[]
for i in range(600000,len(dx)):
    item=row_to_object(process_row(dx.iloc[i]))
    results.append(item)
    print(f"processing row {i} over {(len(dx))}")

In [36]:
results=pd.DataFrame(results)
results.to_csv("abbreviationsList10.csv")

In [ ]:
import pandas as pd 
data=pd.read_csv("abbreviationsList10.csv")
results=[]
for i in range(len(data)):
    item=make_valid_json(data.iloc[i,1])
    results.append(item)
    print(f"processing item {i} over {len(data)}")
results=pd.DataFrame(results)
results.to_csv("fragment11.csv")

In [ ]:
import pandas as pd 
data1=pd.read_json("abbreviationsAnnotated.json")